<a href="https://colab.research.google.com/github/jdmartinezrs/incomesAndBillsProject/blob/main/Trabajo1ParaEntregar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Importar Dataset**

In [5]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
sheet_id ='1Te2gVFrIhMKCPS5WDFTz7XuLQBBG_8CJ'
sheet_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv'

dp = pd.read_csv(sheet_url, header=5)
print(dp.head())



           Fecha  ITEM CODIGO DE EQUIPO Tipo de Vehiculo fecha de revision  \
0            NaN   1.0  VE-DT-CH-XMD284     CARRO TANQUE         5/22/2025   
1            NaN   2.0  VE-DT-FL-XMD588      DOBLETROQUE         5/22/2025   
2            NaN   3.0  VE-DT-FL-SUE755      DOBLETROQUE         5/22/2025   
3  5/22/25 19:21   4.0  VE-DT-FL-WFD547      DOBLETROQUE         5/22/2025   
4  5/22/25 19:21   5.0  VE-DT-IN-TAV909      DOBLETROQUE         5/22/2025   

  Km/ Hr Plan Fecha Ult Pre-Operacional  Dias Pend   Ult Km/Hr Km/Hr Cambio  \
0     10,000                  5/20/2025        2.0    320,832      316,580    
1     10,000                  9/21/2023      609.0    209,543      199,850    
2      8,000                  5/20/2025        2.0    431,842       429,813   
3     10,000                  5/20/2025        2.0    269,735       263,671   
4     10,000                  5/20/2025        2.0    288,404      287,932    

  Fecha Cambio Fecha Prox Cambio Unnamed: 12 Km/Hr Proye

In [11]:
print(dp.isna().sum())

Fecha                         8
ITEM                          4
CODIGO DE EQUIPO              4
Tipo de Vehiculo              4
fecha de revision             4
Km/ Hr Plan                   4
Fecha Ult Pre-Operacional     4
Dias Pend                     4
Ult Km/Hr                     4
Km/Hr Cambio                  4
Fecha Cambio                  4
Fecha Prox Cambio            14
Unnamed: 12                  15
Km/Hr Proyectada              4
Km/Hr Pend                    4
Progreso                      4
Observaciones                32
dtype: int64


In [12]:
for col in dp.columns:
    if dp[col].dtype == 'object':
        dp[col] = dp[col].fillna('N/A')
    else:
        dp[col] = dp[col].fillna(0)

# Mostrar resultado
print(dp.head())

           Fecha  ITEM CODIGO DE EQUIPO Tipo de Vehiculo fecha de revision  \
0            N/A   1.0  VE-DT-CH-XMD284     CARRO TANQUE         5/22/2025   
1            N/A   2.0  VE-DT-FL-XMD588      DOBLETROQUE         5/22/2025   
2            N/A   3.0  VE-DT-FL-SUE755      DOBLETROQUE         5/22/2025   
3  5/22/25 19:21   4.0  VE-DT-FL-WFD547      DOBLETROQUE         5/22/2025   
4  5/22/25 19:21   5.0  VE-DT-IN-TAV909      DOBLETROQUE         5/22/2025   

  Km/ Hr Plan Fecha Ult Pre-Operacional  Dias Pend   Ult Km/Hr Km/Hr Cambio  \
0     10,000                  5/20/2025        2.0    320,832      316,580    
1     10,000                  9/21/2023      609.0    209,543      199,850    
2      8,000                  5/20/2025        2.0    431,842       429,813   
3     10,000                  5/20/2025        2.0    269,735       263,671   
4     10,000                  5/20/2025        2.0    288,404      287,932    

  Fecha Cambio Fecha Prox Cambio Unnamed: 12 Km/Hr Proye

**1.1 Importar las liberias necesarias**

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

**2. Crear un conjunto de datos**

In [16]:
data = {
    'km_hr_pend': dp['Km/Hr Pend'].tolist(),  # Convert column to a list
    'dias_pend': dp['Dias Pend'].tolist(),  # Convert column to a list
    'progreso': dp['Progreso'].tolist()  # Convert column to a list
}

dx = pd.DataFrame(data)
print(dx)

   km_hr_pend  dias_pend progreso
0      5,748         2.0      57%
1        307       609.0       3%
2      5,971         2.0      75%
3      3,936         2.0      39%
4      9,528         2.0      95%
5      1,789       372.0      15%
6     11,446       301.0      95%
7     12,352       397.0     103%
8      7,034       111.0      70%
9      5,675       301.0      47%
10     4,015         2.0      40%
11    12,049       388.0     100%
12    12,006       371.0     100%
13     4,605         2.0      46%
14     9,655       241.0      97%
15     7,087       188.0      71%
16     5,463        31.0      55%
17     3,481         6.0      35%
18     9,160         2.0      92%
19     2,769         2.0      55%
20     5,000        84.0     100%
21     - 429       241.0      -5%
22    12,000         7.0     100%
23     8,555        42.0      71%
24     3,499         2.0      58%
25     2,410        83.0      40%
26     6,274       392.0      90%
27     6,665       462.0      95%
28     3,468  

In [7]:
import numpy as np

In [17]:
import pandas as pd
import numpy as np

# Asegúrate de que los datos sean numéricos
dx['km_hr_pend'] = pd.to_numeric(dx['km_hr_pend'], errors='coerce')
dx['dias_pend'] = pd.to_numeric(dx['dias_pend'], errors='coerce')
dx['progreso'] = pd.to_numeric(dx['progreso'], errors='coerce')

# Definir condiciones de riesgo
conditions = [
    (dx['progreso'] > 90) | (dx['km_hr_pend'] < 0),
    (dx['progreso'] > 70) & (dx['progreso'] <= 90),
    (dx['progreso'] <= 70)
]

choices = ['Alto', 'Medio', 'Bajo']

# Especificar un valor por defecto que también sea string
dx['riesgo'] = np.select(conditions, choices, default='Desconocido')

# Mostrar resultado
print(dx[['km_hr_pend', 'dias_pend', 'progreso', 'riesgo']])



    km_hr_pend  dias_pend  progreso       riesgo
0          NaN        2.0       NaN  Desconocido
1        307.0      609.0       NaN  Desconocido
2          NaN        2.0       NaN  Desconocido
3          NaN        2.0       NaN  Desconocido
4          NaN        2.0       NaN  Desconocido
5          NaN      372.0       NaN  Desconocido
6          NaN      301.0       NaN  Desconocido
7          NaN      397.0       NaN  Desconocido
8          NaN      111.0       NaN  Desconocido
9          NaN      301.0       NaN  Desconocido
10         NaN        2.0       NaN  Desconocido
11         NaN      388.0       NaN  Desconocido
12         NaN      371.0       NaN  Desconocido
13         NaN        2.0       NaN  Desconocido
14         NaN      241.0       NaN  Desconocido
15         NaN      188.0       NaN  Desconocido
16         NaN       31.0       NaN  Desconocido
17         NaN        6.0       NaN  Desconocido
18         NaN        2.0       NaN  Desconocido
19         NaN      

**3.Crear y preparar los datos**
